In [1]:
import os
import cv2
# import uuid
from functions import *
from ultralytics import YOLO
from paddleocr import PaddleOCR
from collections import defaultdict

In [2]:
weight_vehicle_detection_path = "/home/namtt/WorkSpace/MyProjects/Vehicle/weights/vehicle_detection.pt"
tracker_path = "/home/namtt/WorkSpace/MyProjects/Vehicle/weights/tracker/botsort.yaml"
weight_license_plate_detection_path = "/home/namtt/WorkSpace/MyProjects/Vehicle/weights/license_plate_detection.pt"

video_path = "/home/namtt/WorkSpace/MyProjects/Vehicle/data/IMG_4527.MOV"
device = "cpu"
class_names = ["car", "motor", "bus", "truck"]
colors = [
    (0, 0, 255),
    (0, 255, 0),
    (255, 0, 0),
    (0, 255, 255) 
]

model_vehicle = YOLO(weight_vehicle_detection_path)
model_vehicle.eval()
model_license_plate = YOLO(weight_license_plate_detection_path)
model_license_plate.eval()
ocr_model = PaddleOCR(
    use_doc_orientation_classify=False,
    use_doc_unwarping=False,
    use_textline_orientation=False
)

/home/namtt/miniconda3/envs/trafic_detection/lib/python3.10/site-packages/paddle/utils/cpp_extension/extension_utils.py:718: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)
Creating model: ('PP-OCRv5_server_det', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/home/namtt/.paddlex/official_models/PP-OCRv5_server_det`.
Creating model: ('PP-OCRv5_server_rec', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/home/namtt/.paddlex/official_models/PP-OCRv5_server_rec`.


In [3]:
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
w = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
h = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
# output_filename = 'output.mp4'

# fourcc = cv2.VideoWriter_fourcc(*'mp4v') 

# out = cv2.VideoWriter(output_filename, fourcc, fps, (int(w), int(h)))
skip = 0.2
skip_frame = max(1, int(fps*skip))
frame_count = 0
scale_plot = 0.4
scale_process = 0.4

In [ ]:
# create_data = SelectAreaDetect(video_path, scale_plot)
# create_data.create_txt()

: 

In [ ]:
mask = gen_mask("IMG_4527/IMG_4527.txt", "IMG_4527/IMG_4527.png", scale_process)
lass_detect = defaultdict()
while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame_count += 1
    if frame_count % skip_frame != 0:
        continue
    frame, frame_pr = process_image(frame, mask, scale_process)
    vehicle_result = model_vehicle.track(
        frame_pr, 
        conf=0.75, 
        iou=0.7,
        tracker=tracker_path,
        device=device,
        persist=True,
        verbose=False 
    )[0]
    for bbox in vehicle_result.boxes:
        if bbox.id is not None:
            x1, y1, x2, y2 = bbox.xyxy[0].cpu().numpy().astype(int)
            id_ = bbox.id[0].cpu().numpy().astype(int)
            cls = bbox.cls[0].cpu().numpy().astype(int)
            color = colors[cls]
            obj = frame[y1:y2, x1:x2]
            label = ""
            try:
                license_plate_result = model_license_plate.predict(obj, conf=0.8, verbose=False, iou=0.8, device=device)[0].boxes[0]
                lx1, ly1, lx2, ly2 = license_plate_result.xyxy[0].cpu().numpy().astype(int)
                license = obj[ly1:ly2, lx1:lx2]
                ocr_result = ocr_model.predict(license)[0]
                conf = 0
                if ocr_result['rec_texts'] is not None:
                    for i in len(ocr_result['rec_texts']):
                        label += ocr_result['rec_texts'][i]
                        conf += ocr_result['rec_scores'][i]
                    conf /= len(ocr_result['rec_texts'])
                    label = post_process_license_plate(label)
                if id_ not in lass_detect:
                    lass_detect[id_] = {
                        "label": label,
                        "conf": conf
                    }
                    cv2.putText(frame, f"{label}", (x1, y1+3), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)
                else:
                    if lass_detect[id_]['conf'] < conf:
                        lass_detect[id_]['conf'] = conf
                        lass_detect[id_]['label'] = label
                        cv2.putText(frame, f"{label}", (x1, y1+3), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)
            except:
                cv2.putText(frame, f"Can't OCR", (x1, y1+3), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)
                pass
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
    # out.write(frame)
    # frame
    cv2.imshow("Image", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
# out.release()
cv2.destroyAllWindows()

qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/namtt/miniconda3/envs/trafic_detection/lib/python3.10/site-packages/cv2/qt/plugins"


In [ ]:
cap.release()
# out.release()
cv2.destroyAllWindows()